In [6]:
import numpy as np
from qiskit_finance.circuit.library import LogNormalDistribution
from scipy.integrate import quad
from scipy.stats import lognorm

# Vanilla Call options

In [13]:
# number of qubits to represent the uncertainty for our probability distribution
num_uncertainty_qubits = 3

# parameters for considered random distribution
S = 2.0  # initial spot price
vol = 0.4  # volatility of 40%
r = 0.05  # annual interest rate of 4%
T = 40 / 365  # 40 days to maturity

# resulting parameters for log-normal distribution
mu = (r - 0.5 * vol**2) * T + np.log(S)
sigma = vol * np.sqrt(T)
mean = np.exp(mu + sigma**2 / 2)
variance = (np.exp(sigma**2) - 1) * np.exp(2 * mu + sigma**2)
stddev = np.sqrt(variance)

# lowest and highest value considered for the spot price; in between, an equidistant discretization is considered.
low = np.maximum(0, mean - 3 * stddev)
high = mean + 3 * stddev

uncertainty_model = LogNormalDistribution(
    num_uncertainty_qubits, mu=mu, sigma=sigma**2, bounds=(low, high)
)

scale = np.exp(mu) 

def call_payoff(S_T, K, r, T):
    return np.exp(-r * T) * max(S_T - K, 0)

strike_price = 1.9
# Integrating the call payoff function against the lognormal PDF from K to infinity
continuous_expected_value, _ = quad(lambda S_T: call_payoff(S_T, strike_price, r, T) * lognorm.pdf(S_T, s=sigma, scale=scale), strike_price, np.inf)

payoff = np.maximum(uncertainty_model.values - strike_price, 0)
discrete_expected_value = np.dot(uncertainty_model.probabilities, payoff)

print("Exact value:     \t{}".format(continuous_expected_value))
print("Discrete value: \t{}".format(discrete_expected_value))
print("Relative error:  \t{} %".format(((1 - discrete_expected_value / continuous_expected_value)*100)))

Exact value:     	0.16713122937760183
Discrete value: 	0.16027265994646528
Relative error:  	4.1037030940763835 %


# Basket Call

In [20]:
import numpy as np
from package.option_pricing import OptionPricing, OptionParams
## define options
import numpy as np
from qiskit_finance.circuit.library import LogNormalDistribution
# number of qubits per dimension to represent the uncertainty
num_uncertainty_qubits = 3

# parameters for considered random distribution
S = 2.0  # initial spot price
vol = 0.4  # volatility of 40%
r = 0.04  # annual interest rate of 4%
T = 40 / 365  # 40 days to maturity

# resulting parameters for log-normal distribution
mu = (r - 0.5 * vol**2) * T + np.log(S)
sigma = vol * np.sqrt(T)
mean = np.exp(mu + sigma**2 / 2)
variance = (np.exp(sigma**2) - 1) * np.exp(2 * mu + sigma**2)
stddev = np.sqrt(variance)

# lowest and highest value considered for the spot price; in between, an equidistant discretization is considered.
low = np.maximum(0, mean - 3 * stddev)
high = mean + 3 * stddev

# map to higher dimensional distribution
# for simplicity assuming dimensions are independent and identically distributed)
dimension = 2
num_qubits = [num_uncertainty_qubits] * dimension
low = low * np.ones(dimension)
high = high * np.ones(dimension)
mu = mu * np.ones(dimension)
cov = sigma**2 * np.eye(dimension) # covariance matrix

# construct circuit
uncertainty_model = LogNormalDistribution(num_qubits=num_qubits, mu=mu, sigma=cov, bounds=list(zip(low, high)))

sum_values = np.sum(uncertainty_model.values, axis=1)
discrete_expected_value = np.dot(
    uncertainty_model.probabilities[sum_values >= strike_price], 
    sum_values[sum_values >= strike_price] - strike_price
)

In [21]:
import numpy as np

def expected_value_basket_call(asset1_params, asset2_params, covariance_matrix, K, n_simulations=100000):
    """
    Calculate the expected value of a basket call option using Monte Carlo simulation.

    Parameters:
    - asset1_params: dict with 'S', 'vol', 'r', 'T' for the first asset.
    - asset2_params: dict with 'S', 'vol', 'r', 'T' for the second asset.
    - covariance_matrix: 2x2 matrix representing the covariance between the assets.
    - K: Strike price of the basket call option.
    - n_simulations: Number of simulations to run (default is 100,000).

    Returns:
    - Expected value of the basket call option's payoff.
    """
    np.random.seed(0)  # For reproducibility
    
    # Unpack parameters
    S1, vol1, r1, T1 = asset1_params['S'], asset1_params['vol'], asset1_params['r'], asset1_params['T']
    S2, vol2, r2, T2 = asset2_params['S'], asset2_params['vol'], asset2_params['r'], asset2_params['T']
    
    # Ensure same r and T for both assets (for simplicity)
    r, T = r1, T1
    
    # Generate random numbers with the given covariance
    z = np.random.multivariate_normal([0, 0], covariance_matrix, size=n_simulations)
    
    # Simulate future values for both assets
    x1_T = S1 * np.exp((r - 0.5 * vol1**2) * T + vol1 * np.sqrt(T) * z[:, 0])
    x2_T = S2 * np.exp((r - 0.5 * vol2**2) * T + vol2 * np.sqrt(T) * z[:, 1])
    
    # Compute payoffs for each simulation
    payoffs = np.maximum(x1_T + x2_T - K, 0)
    
    # Compute the expected payoff and discount it back to present value
    expected_value = np.mean(payoffs) * np.exp(-r * T)
    
    return expected_value

# Example parameters and covariance matrix
asset1_params = {'S': 2.0, 'vol': 0.4, 'r': 0.04, 'T': 40/365}
asset2_params = {'S': 2.0, 'vol': 0.4, 'r': 0.04, 'T': 40/365}
covariance_matrix = np.array([[0.16, 0.064], [0.064, 0.16]])  # Example covariance matrix
K = 1.9  # Strike price

# Calculate the expected value of the basket call
expected_value = expected_value_basket_call(asset1_params, asset2_params, covariance_matrix, K)
print(f"Exact Expected value: {expected_value}")
print(f"Discrete Expected value: {discrete_expected_value}")
print(f"Relative error: {(1 - discrete_expected_value / expected_value) * 100} %")


Exact Expected value: 2.078926671761519
Discrete Expected value: 2.114150038090828
Relative error: -1.6943053743912584 %
